In [1]:
!pip install -U "langgraph>=0.2.0" \
               "langchain>=0.2.11" "langchain-core>=0.2.33" \
               "langchain-community>=0.2.11" "langchain-openai>=0.2.1" \
               "faiss-cpu" "langchain-text-splitters"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.8/473.8 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.3/208.3 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing ins

In [3]:
from typing_extensions import TypedDict, Annotated
import operator
from langgraph.graph import StateGraph, START, END   # ← importar START y END
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

class AgentState(TypedDict):
    messages: Annotated[list, operator.add]

# Modelo del agente
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

def assistant_node(state: AgentState) -> AgentState:
    # Llamar al modelo usando TODO el historial
    response = llm.invoke(state["messages"])
    return {"messages": [response]}

# Construcción del grafo
builder = StateGraph(AgentState)

builder.add_node("assistant", assistant_node)

# Transiciones del flujo
builder.add_edge(START, "assistant")
builder.add_edge("assistant", END)

# Compilación del grafo
graph = builder.compile()

# Estado inicial
initial_state = {
    "messages": [HumanMessage(content="Probando mi primer agente LangGraph :)")]
}

# Ejecución del agente
result = graph.invoke(initial_state)
print(result["messages"][-1].content)


/usr/local/lib/python3.12/dist-packages/pydantic/v1/main.py:1054: UserWarning: LangSmith now uses UUID v7 for run and trace identifiers. This warning appears when passing custom IDs. Please use: from langsmith import uuid7
            id = uuid7()
Future versions will require UUID v7.
  input_data = validator(cls_, input_data)


¡Eso suena emocionante! LangGraph es una herramienta poderosa para trabajar con modelos de lenguaje y crear agentes conversacionales. Si tienes alguna pregunta específica o necesitas ayuda con algo relacionado con tu agente, no dudes en preguntar. Estoy aquí para ayudarte. ¿Qué estás tratando de lograr con tu agente LangGraph?


In [4]:
from typing import Optional
from typing_extensions import TypedDict, Annotated
import operator

class AgentState(TypedDict):
    messages: Annotated[list, operator.add]
    summary: Optional[str]

In [5]:
from typing import Optional
from typing_extensions import TypedDict, Annotated
import operator

class AgentState(TypedDict):
    messages: Annotated[list, operator.add]
    summary: Optional[str]

# Estado inicial
initial_state = {
    "messages": [],
    "summary": None
}


In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

# Corpus mínimo (podés cambiarlo por algo de tu dominio)
raw_docs = [
    "LangGraph permite orquestar agentes como grafos de estado.",
    "RAG combina recuperación + generación para mejorar grounding.",
    "LangChain y LangGraph se integran con OpenAI, HuggingFace y más."
]

docs = [Document(page_content=t) for t in raw_docs]

# Split en chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(docs)

# Vector store FAISS
emb = OpenAIEmbeddings()
vs = FAISS.from_documents(chunks, embedding=emb)
retriever = vs.as_retriever(search_kwargs={"k": 3})

In [7]:
from langchain_core.tools import tool

@tool
def rag_search(question: str) -> str:
    """
    Tool de recuperación (RAG).
    Recibe una pregunta como string y devuelve texto relevante recuperado
    desde el vector store. Ideal para que el LLM use como contexto.
    """
    docs = retriever.vectorstore.similarity_search(
        question,
        k=retriever.search_kwargs.get("k", 3),
    )
    context = "\n\n".join(d.page_content for d in docs)
    if not context:
        return "No se encontró información relevante en el corpus."
    return context


In [8]:
from datetime import datetime
from langchain_core.tools import tool

FAKE_ORDERS = {
    "ABC123": "En preparación",
    "XYZ999": "Entregado",
}

@tool
def get_order_status(order_id: str) -> str:
    """
    Devuelve el estado de un pedido ficticio dado su ID.
    Esta tool simula un servicio de soporte que consulta pedidos.
    """
    status = FAKE_ORDERS.get(order_id)
    if status is None:
        return f"No encontré el pedido {order_id}."
    return f"Estado actual del pedido {order_id}: {status}"

@tool
def get_utc_time(_: str = "") -> str:
    """
    Devuelve la hora actual en formato ISO (UTC).
    """
    return datetime.utcnow().isoformat()


In [9]:
from langgraph.prebuilt import ToolNode
from langchain_core.messages import AIMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END # Importaciones necesarias

# 1) Lista de tools (Definidas en pasos anteriores)
tools = [rag_search, get_order_status, get_utc_time]

# 2) LLM con tools
# Usamos gpt-4o-mini que es eficiente y soporta function calling
llm_with_tools = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind_tools(tools)

def assistant_node(state: AgentState) -> AgentState:
    """
    Nodo de reasoning: El cerebro del agente.
    Decide si responde directo (texto) o solicita usar una tool (tool_call).
    """
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

# 3) Nodo que ejecuta efectivamente las tools
# Este nodo toma el output del assistant, busca la función correspondiente y la ejecuta.
tool_node = ToolNode(tools)

# Router para decidir el flujo
def route_from_assistant(state: AgentState) -> str:
    last = state["messages"][-1]
    # Si el LLM generó una llamada a herramienta, vamos al nodo "tools"
    if isinstance(last, AIMessage) and last.tool_calls:
        return "tools"
    # Si no, terminamos el flujo
    return END

# 🎯 Construcción del Grafo
builder = StateGraph(AgentState)

# Nodos
builder.add_node("assistant", assistant_node)
builder.add_node("tools", tool_node)

# Aristas
builder.add_edge(START, "assistant")

# Decisión condicional
builder.add_conditional_edges(
    "assistant",
    route_from_assistant,
    {
        "tools": "tools",
        END: END,
    }
)

# Ciclo de retorno (La clave de LangGraph)
builder.add_edge("tools", "assistant")

# Compilación
graph = builder.compile()

print("✅ Grafo compilado exitosamente.")

✅ Grafo compilado exitosamente.


In [10]:
# Prueba
consulta = "Hola, necesito saber el estado del pedido ABC123 y también quiero saber qué es LangGraph según tus documentos."

for event in graph.stream({"messages": [("user", consulta)]}):
    for value in event.values():
        print("--- Paso del Agente ---")
        print(value["messages"][-1].content)

--- Paso del Agente ---

--- Paso del Agente ---
LangGraph permite orquestar agentes como grafos de estado.

LangChain y LangGraph se integran con OpenAI, HuggingFace y más.

RAG combina recuperación + generación para mejorar grounding.
--- Paso del Agente ---
El estado actual del pedido **ABC123** es: **En preparación**.

En cuanto a **LangGraph**, permite orquestar agentes como grafos de estado. LangChain y LangGraph se integran con OpenAI, HuggingFace y más. Además, RAG combina recuperación y generación para mejorar el grounding.


In [11]:
from langchain_core.messages import HumanMessage

state = {
    "messages": [
        HumanMessage(content="Hola, ¿qué es LangGraph en pocas palabras?")
    ],
    "summary": None
}

result = graph.invoke(state)
print("Respuesta 1:", result["messages"][-1].content)

Respuesta 1: LangGraph es una herramienta que permite orquestar agentes como grafos de estado. Se integra con plataformas como LangChain, OpenAI y HuggingFace, y utiliza un enfoque de Recuperación y Generación (RAG) para mejorar el grounding en las interacciones.


In [12]:
from copy import deepcopy

state2 = deepcopy(result)
state2["messages"].append(HumanMessage(content="Usá tu base de conocimiento y decime qué es RAG."))

result2 = graph.invoke(state2)
print("Respuesta 2:", result2["messages"][-1].content)

Respuesta 2: RAG, que significa Recuperación y Generación, es un enfoque que combina la recuperación de información con la generación de texto para mejorar el grounding en las interacciones. Esto permite que los modelos de lenguaje utilicen información relevante de manera más efectiva al generar respuestas.


In [13]:
for event in graph.stream(state2, stream_mode="values"):
    msgs = event["messages"]
    print("Último mensaje:", msgs[-1].type, "→", msgs[-1].content if hasattr(msgs[-1], "content") else msgs[-1])

Último mensaje: human → Usá tu base de conocimiento y decime qué es RAG.
Último mensaje: ai → 
Último mensaje: tool → RAG combina recuperación + generación para mejorar grounding.

LangGraph permite orquestar agentes como grafos de estado.

LangChain y LangGraph se integran con OpenAI, HuggingFace y más.
Último mensaje: ai → RAG, que significa Recuperación y Generación, es un enfoque que combina la recuperación de información con la generación de texto para mejorar el grounding en las interacciones. Esto permite que los modelos de lenguaje utilicen información relevante de fuentes externas para generar respuestas más precisas y contextualmente adecuadas.


In [16]:
# --- Importaciones necesarias ---
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langchain_core.messages import AIMessage, SystemMessage
from langchain_openai import ChatOpenAI

# --- 1. Definición de Nodos y Lógica ---

# (Asegúrate de tener las tools y AgentState definidos en celdas anteriores)

# Nodo Asistente (Cerebro)
def assistant_node(state: AgentState) -> AgentState:
    # Inyectamos el resumen en el contexto si existe
    summary = state.get("summary", "")
    messages = state["messages"]

    if summary:
        # Añadimos un mensaje de sistema con el resumen al inicio
        # (Truco: esto le da contexto sin gastar tokens en mensajes viejos si los hubiéramos borrado)
        system_message = SystemMessage(content=f"Resumen de la conversación anterior: {summary}")
        messages = [system_message] + messages

    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

# Nodo de Memoria (El nuevo componente)
def memory_node(state: AgentState) -> AgentState:
    summary_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    current_summary = state.get("summary", "") or "Nada aún."

    # Tomamos los últimos mensajes para actualizar el resumen
    recent_messages = state["messages"][-5:]
    conversation_str = "\n".join([f"{m.type}: {m.content}" for m in recent_messages])

    prompt = f"""
    Resumen actual: {current_summary}
    Nueva conversación: {conversation_str}

    Genera un nuevo resumen actualizado conciso (max 3 líneas).
    """
    response = summary_llm.invoke(prompt)
    return {"summary": response.content}

# Nodo de Herramientas (Brazos)
tool_node = ToolNode(tools)

# Router (Semáforo)
def route_from_assistant(state: AgentState) -> str:
    last = state["messages"][-1]
    if isinstance(last, AIMessage) and last.tool_calls:
        return "tools"
    return END

# --- 2. Construcción Limpia del Grafo ---

# Reiniciamos el builder desde cero
builder = StateGraph(AgentState)

# Añadimos TODOS los nodos
builder.add_node("assistant", assistant_node)
builder.add_node("tools", tool_node)
builder.add_node("memory", memory_node)  # <--- Nuevo nodo

# Definimos el flujo (Edges)

# Inicio -> Asistente
builder.add_edge(START, "assistant")

# Asistente -> ¿Tools o Fin?
builder.add_conditional_edges(
    "assistant",
    route_from_assistant,
    {
        "tools": "tools",
        END: END
    }
)

# --- EL CAMBIO CLAVE EN EL FLUJO ---
# Antes era: tools -> assistant
# Ahora es:  tools -> memory -> assistant

builder.add_edge("tools", "memory")      # Después de actuar, resumimos
builder.add_edge("memory", "assistant")  # Después de resumir, volvemos al asistente

# Compilación final
graph = builder.compile()

print("✅ Grafo con ciclo de memoria compilado correctamente.")

✅ Grafo con ciclo de memoria compilado correctamente.


In [17]:
from langchain_core.messages import HumanMessage

# 1. Definimos una consulta que obligue a usar una herramienta
# (Esto activará el camino: Assistant -> Tools -> Memory -> Assistant)
inputs = {
    "messages": [HumanMessage(content="Hola, ¿podrías verificar el estado del pedido ABC123?")],
    "summary": ""  # Empezamos con la memoria vacía
}

# 2. Ejecutamos el grafo
# recursion_limit es una seguridad para evitar bucles infinitos si la lógica fallara
result = graph.invoke(inputs, config={"recursion_limit": 10})

# 3. Inspeccionamos los resultados
print(f"🤖 Respuesta Final del Asistente:\n{result['messages'][-1].content}")
print("\n" + "="*30 + "\n")
print(f"🧠 Memoria a Largo Plazo (Summary) Generada:\n{result.get('summary', '⚠️ No se generó resumen')}")

🤖 Respuesta Final del Asistente:
El estado del pedido ABC123 es: En preparación.


🧠 Memoria a Largo Plazo (Summary) Generada:
Estado del pedido ABC123: En preparación.


In [18]:
import gradio as gr
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

def format_chat_history(messages):
    """Convierte el historial de LangChain a formato de chat de Gradio [[user, bot], ...]"""
    history = []
    temp_user_msg = None

    for msg in messages:
        if isinstance(msg, HumanMessage):
            temp_user_msg = msg.content
        elif isinstance(msg, AIMessage) and msg.content:
            # Solo mostramos respuestas que tengan contenido de texto (ignoramos tool calls vacíos)
            if temp_user_msg:
                history.append([temp_user_msg, msg.content])
                temp_user_msg = None
            else:
                # Caso raro: AI habla sin prompt humano inmediato (continuación)
                history.append([None, msg.content])
    return history

def extract_debug_info(messages):
    """Busca qué herramientas se usaron en la última interacción"""
    tools_used = []
    # Recorremos hacia atrás buscando ToolMessages recientes
    for msg in reversed(messages):
        if isinstance(msg, HumanMessage):
            break # Paramos al llegar al último input del usuario
        if isinstance(msg, ToolMessage):
            tools_used.append(f"🔧 {msg.name} (ID: {msg.tool_call_id[:5]}...)")
            # Opcional: Mostrar el resultado de la tool
            # tools_used.append(f"   -> Resultado: {msg.content[:50]}...")

    if not tools_used:
        return "No se usaron herramientas en este turno."
    return "\n".join(reversed(tools_used))

def run_agent(input_text, state):
    # 1. Inicializar estado si está vacío
    if not state:
        state = {"messages": [], "summary": ""}

    # 2. Agregar mensaje del usuario
    state["messages"].append(HumanMessage(content=input_text))

    # 3. Invocar el Grafo
    # recursion_limit alto por si el agente entra en bucles de pensamiento
    result = graph.invoke(state, config={"recursion_limit": 10})

    # 4. Procesar salidas para la UI
    chat_history = format_chat_history(result["messages"])
    tools_log = extract_debug_info(result["messages"])
    current_summary = result.get("summary", "Sin resumen aún.")

    return chat_history, result, tools_log, current_summary

# --- Interfaz Gradio ---
with gr.Blocks(theme=gr.themes.Soft()) as ui:
    gr.Markdown("# 🤖 Agente ReAct con Memoria y RAG")
    gr.Markdown("Prueba interactiva del TP Final - UT4")

    with gr.Row():
        # Columna Izquierda: Chat
        with gr.Column(scale=2):
            chatbot = gr.Chatbot(height=500, label="Conversación")
            msg = gr.Textbox(label="Tu mensaje", placeholder="Preguntá por pedidos (ABC123) o sobre LangGraph...")
            clear = gr.Button("Limpiar Conversación")

        # Columna Derecha: Debug y Estado Interno
        with gr.Column(scale=1):
            gr.Markdown("### 🧠 Estado Interno del Agente")
            tools_output = gr.Textbox(label="Tools Usadas (Último Turno)", interactive=False)
            summary_output = gr.TextArea(label="Memoria a Largo Plazo (Summary)", interactive=False)

            # Botón para inspeccionar el estado crudo (opcional)
            state_display = gr.JSON(label="Estado Crudo (Full State)", visible=False)

    # Estado de la sesión (persiste entre interacciones)
    agent_state = gr.State()

    # Eventos
    msg.submit(
        run_agent,
        inputs=[msg, agent_state],
        outputs=[chatbot, agent_state, tools_output, summary_output]
    )

    # Limpiar input después de enviar
    msg.submit(lambda: "", None, msg)

    # Botón de limpieza
    def clear_memory():
        return None, [], "", "Memoria reiniciada."

    clear.click(clear_memory, outputs=[agent_state, chatbot, tools_output, summary_output])

ui.launch(debug=True)

/tmp/ipython-input-2490267038.py:65: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=500, label="Conversación")


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://383e0f962055a8e0ca.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://383e0f962055a8e0ca.gradio.live


In [19]:
import operator
from typing import Optional, List, Annotated
from typing_extensions import TypedDict
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode

# --- 1. Configuración del Dominio (RAG) ---
# Corpus de conocimiento (FAQs del curso basadas en tus PDFs)
texts_curso = [
    "La evaluación del curso consiste en un portafolio de proyectos (60%) y una defensa oral final (10%).",
    "Para la aprobación directa se requiere una calificación de B o superior.",
    "El curso cubre unidades de Machine Learning Clásico, Deep Learning, Computer Vision, NLP y MLOps.",
    "Las herramientas utilizadas incluyen Python, Pandas, Scikit-learn, PyTorch y LangChain.",
    "La metodología es aprendizaje basado en equipos y proyectos, con preparación previa (Flipped Classroom).",
    "La defensa final del curso está programada para el 02/12.",
    "El profesor a cargo es el Ing. Juan Francisco Kurucz."
]

# Construcción del índice vectorial
docs = [Document(page_content=t) for t in texts_curso]
splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
chunks = splitter.split_documents(docs)
vectorstore = FAISS.from_documents(chunks, OpenAIEmbeddings())
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

# --- 2. Definición de Tools ---

@tool
def consultar_reglamento(pregunta: str) -> str:
    """Usa esta tool para responder dudas sobre el funcionamiento, evaluación, fechas o contenido del curso."""
    docs = retriever.invoke(pregunta)
    return "\n".join([d.page_content for d in docs]) if docs else "No encontré información en el reglamento."

# Base de datos ficticia de alumnos
DB_ALUMNOS = {
    "A001": {"nombre": "Ana", "estado": "Regular", "entregas_pendientes": 0},
    "A002": {"nombre": "Beto", "estado": "Condicional", "entregas_pendientes": 2},
}

@tool
def ver_estado_alumno(matricula: str) -> str:
    """Consulta el estado académico de un alumno dado su ID de matrícula (ej: A001)."""
    alumno = DB_ALUMNOS.get(matricula)
    if not alumno:
        return "Matrícula no encontrada."
    return f"Alumno: {alumno['nombre']} | Estado: {alumno['estado']} | Pendientes: {alumno['entregas_pendientes']}"

tools = [consultar_reglamento, ver_estado_alumno]

# --- 3. Definición del Estado y Grafo ---

class AgentState(TypedDict):
    messages: Annotated[list, operator.add]
    summary: Optional[str]

# Modelo y Nodos
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind_tools(tools)

def assistant_node(state: AgentState) -> AgentState:
    # Inyectamos contexto de memoria si existe
    msgs = state["messages"]
    if state.get("summary"):
        msgs = [SystemMessage(content=f"Resumen previo: {state['summary']}")] + msgs
    return {"messages": [llm.invoke(msgs)]}

def memory_node(state: AgentState) -> AgentState:
    # Resumidor ligero
    summary_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    last_msgs = state["messages"][-4:] # Miramos lo reciente
    conversation = "\n".join([f"{m.type}: {m.content}" for m in last_msgs])
    curr_summary = state.get("summary") or ""

    prompt = f"Actualiza este resumen: '{curr_summary}' con esto nuevo: '{conversation}'. Sé breve."
    new_summary = summary_llm.invoke(prompt).content
    return {"summary": new_summary}

def router(state: AgentState) -> str:
    last = state["messages"][-1]
    if last.tool_calls:
        return "tools"
    return END

# Armado del Grafo
builder = StateGraph(AgentState)
builder.add_node("assistant", assistant_node)
builder.add_node("tools", ToolNode(tools))
builder.add_node("memory", memory_node)

builder.add_edge(START, "assistant")
builder.add_conditional_edges("assistant", router, {"tools": "tools", END: END})
builder.add_edge("tools", "memory")
builder.add_edge("memory", "assistant")

agent = builder.compile()

# --- 4. Ejecución de las 3 Pruebas ---

casos_prueba = [
    "1. Consulta RAG: ¿Cómo se aprueba el curso?",
    "2. Consulta Estado: ¿Cuál es la situación del alumno A002?",
    "3. Mixta: Soy el alumno A001. ¿Tengo entregas pendientes? Y recordame cuándo es la defensa final."
]

print("🏁 INICIANDO PRUEBAS DEL DESAFÍO INTEGRADOR 🏁\n")

for i, prompt in enumerate(casos_prueba):
    print(f"--- PRUEBA {prompt} ---")
    inputs = {"messages": [HumanMessage(content=prompt)], "summary": ""}

    # Ejecutamos
    for event in agent.stream(inputs):
        for node_name, value in event.items():
            if node_name == "assistant":
                last_msg = value["messages"][-1]
                if last_msg.tool_calls:
                    print(f"🔧 Tool invocada: {last_msg.tool_calls[0]['name']}")
                else:
                    print(f"🤖 Respuesta Final: {last_msg.content}")
    print("\n" + "="*40 + "\n")

🏁 INICIANDO PRUEBAS DEL DESAFÍO INTEGRADOR 🏁

--- PRUEBA 1. Consulta RAG: ¿Cómo se aprueba el curso? ---
🔧 Tool invocada: consultar_reglamento
🤖 Respuesta Final: Para aprobar el curso, se requiere un portafolio de proyectos que representa el 60% de la evaluación y una defensa oral final que cuenta por el 10%. Además, es necesario obtener una calificación de B o superior para aprobar directamente.


--- PRUEBA 2. Consulta Estado: ¿Cuál es la situación del alumno A002? ---
🔧 Tool invocada: ver_estado_alumno
🤖 Respuesta Final: El alumno A002, Beto, se encuentra en estado condicional y tiene 2 pendientes.


--- PRUEBA 3. Mixta: Soy el alumno A001. ¿Tengo entregas pendientes? Y recordame cuándo es la defensa final. ---
🔧 Tool invocada: ver_estado_alumno
🤖 Respuesta Final: Como alumno A001, no tienes entregas pendientes y tu estado es regular. La defensa final está programada para el 02/12. La evaluación del curso consiste en un portafolio de proyectos (60%) y una defensa oral final (10%).

